# HUD Fair Market Rents - Quick Start Guide

This notebook demonstrates how to use the **HUDFMRConnector** to access HUD Fair Market Rent data and perform affordability analysis.

**⚠️ API Key Required:**
This notebook requires a HUD USER API key to access live data. 
- **Get your free API key:** [HUD USER Registration](https://www.huduser.gov/hudapi/public/register?comingfrom=2,3)
- Set it as environment variable: `HUD_API_KEY=your_token_here`
- Or pass directly to connector: `HUDFMRConnector(api_key='your_token')`

**Data Sources:**
- Fair Market Rents (FMR) by bedroom count (0BR-4BR)
- Income limits (very low, low, median)
- Small Area FMRs (ZIP code level)

**Use Cases:**
- Housing affordability analysis
- Rental market comparisons
- Income qualification calculations

---

*© 2025 KR-Labs. Licensed under Apache-2.0.*

## 1. Setup and Import

In [ ]:
from krl_data_connectors.housing import HUDFMRConnector
import pandas as pd
import os

# Initialize connector with API key from environment
# To set: export HUD_API_KEY='your_token_here'
api_key = os.getenv('HUD_API_KEY')

if not api_key:
    print("⚠️  No HUD_API_KEY found in environment variables")
    print("   Get your free API key: https://www.huduser.gov/hudapi/public/register?comingfrom=2,3")
    print("   Then run: export HUD_API_KEY='your_token_here'")
    api_key = input("\nOr paste your API key here (or press Enter to skip): ").strip() or None

hud = HUDFMRConnector(api_key=api_key)

if api_key:
    print("✓ HUD FMR connector initialized with API key")
else:
    print("⚠️  HUD FMR connector initialized WITHOUT API key")
    print("   You can load data from downloaded files instead")

## 2. Get State FMRs via API

Retrieve Fair Market Rents for a state.

In [ ]:
if 'ri_fmrs' in locals() and not ri_fmrs.empty:
    # Create affordability matrix
    income_levels = [30000, 40000, 50000, 60000, 75000, 100000]
    avg_2br_fmr = ri_fmrs['fmr_2br'].mean()
    
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=(
            'Affordability by Income Level (2BR)',
            'Rent Burden Analysis'
        ),
        specs=[[{'type': 'bar'}, {'type': 'scatter'}]]
    )
    
    # Calculate affordability metrics
    affordable = []
    max_affordable_rents = []
    rent_burdens = []
    
    for income in income_levels:
        max_rent = (income / 12) * 0.30
        max_affordable_rents.append(max_rent)
        affordable.append(max_rent >= avg_2br_fmr)
        rent_burdens.append((avg_2br_fmr / (income / 12)) * 100)
    
    # 1. Affordability bar chart
    colors = ['green' if a else 'red' for a in affordable]
    fig.add_trace(
        go.Bar(
            x=[f"${i//1000}k" for i in income_levels],
            y=max_affordable_rents,
            name='Max Affordable Rent',
            marker_color=colors,
            text=[f"${r:.0f}" for r in max_affordable_rents],
            textposition='auto'
        ),
        row=1, col=1
    )
    
    # Add average FMR line
    fig.add_trace(
        go.Scatter(
            x=[f"${i//1000}k" for i in income_levels],
            y=[avg_2br_fmr] * len(income_levels),
            mode='lines',
            name=f'Avg 2BR FMR (${avg_2br_fmr:.0f})',
            line=dict(color='red', dash='dash', width=3)
        ),
        row=1, col=1
    )
    
    # 2. Rent burden percentage
    fig.add_trace(
        go.Scatter(
            x=[f"${i//1000}k" for i in income_levels],
            y=rent_burdens,
            mode='lines+markers',
            name='Rent Burden %',
            line=dict(color='darkblue', width=3),
            marker=dict(size=10),
            text=[f"{b:.1f}%" for b in rent_burdens],
            textposition='top center'
        ),
        row=1, col=2
    )
    
    # Add 30% guideline
    fig.add_trace(
        go.Scatter(
            x=[f"${i//1000}k" for i in income_levels],
            y=[30] * len(income_levels),
            mode='lines',
            name='30% Guideline',
            line=dict(color='green', dash='dash', width=2)
        ),
        row=1, col=2
    )
    
    # Update layout
    fig.update_xaxes(title_text="Annual Income", row=1, col=1)
    fig.update_yaxes(title_text="Monthly Rent ($)", row=1, col=1)
    fig.update_xaxes(title_text="Annual Income", row=1, col=2)
    fig.update_yaxes(title_text="Rent Burden (%)", row=1, col=2)
    
    fig.update_layout(
        title_text=f"Housing Affordability Analysis - RI Average 2BR FMR: ${avg_2br_fmr:.0f}",
        height=500,
        showlegend=True,
        template='plotly_white'
    )
    
    fig.show()
    
    # Print summary
    print("\n📊 Affordability Summary:")
    print(f"Average 2BR FMR: ${avg_2br_fmr:.2f}")
    print(f"\nIncome needed for affordability (30% rule): ${(avg_2br_fmr * 12 / 0.30):,.0f}/year")
    affordable_count = sum(affordable)
    print(f"Income levels tested: {len(income_levels)}")
    print(f"Affordable at: {affordable_count}/{len(income_levels)} income levels")
else:
    print("⚠️  No data available. Please run the previous cell to fetch FMR data.")

## 4. Affordability Analysis Visualization

Analyze housing affordability across income levels.

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

if 'ri_fmrs' in locals() and not ri_fmrs.empty:
    # Create 2x2 dashboard
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'FMR Distribution by Bedroom Count',
            'Geographic Distribution (2BR FMRs)',
            'Metro vs County Comparison',
            'FMR Statistics Summary'
        ),
        specs=[
            [{'type': 'box'}, {'type': 'bar'}],
            [{'type': 'bar'}, {'type': 'table'}]
        ],
        vertical_spacing=0.12,
        horizontal_spacing=0.1
    )
    
    # 1. Box plot - FMR distribution by bedroom count
    bedroom_cols = ['fmr_0br', 'fmr_1br', 'fmr_2br', 'fmr_3br', 'fmr_4br']
    for col in bedroom_cols:
        fig.add_trace(
            go.Box(
                y=ri_fmrs[col],
                name=col.replace('fmr_', '').upper(),
                marker_color='lightblue'
            ),
            row=1, col=1
        )
    
    # 2. Bar chart - Top areas by 2BR FMR
    top_areas = ri_fmrs.nlargest(10, 'fmr_2br')
    area_names = top_areas.apply(
        lambda x: x['county_name'] if pd.notna(x['county_name']) else x['metro_name'],
        axis=1
    )
    fig.add_trace(
        go.Bar(
            x=top_areas['fmr_2br'],
            y=area_names,
            orientation='h',
            marker_color='coral',
            text=top_areas['fmr_2br'],
            textposition='auto'
        ),
        row=1, col=2
    )
    
    # 3. Metro vs County comparison
    metros = ri_fmrs[ri_fmrs['county_name'].isna()]
    counties = ri_fmrs[ri_fmrs['county_name'].notna()]
    
    fig.add_trace(
        go.Bar(
            name='Metro Areas',
            x=['0BR', '1BR', '2BR', '3BR', '4BR'],
            y=[metros[col].mean() for col in bedroom_cols],
            marker_color='lightgreen'
        ),
        row=2, col=1
    )
    fig.add_trace(
        go.Bar(
            name='Counties/Towns',
            x=['0BR', '1BR', '2BR', '3BR', '4BR'],
            y=[counties[col].mean() for col in bedroom_cols],
            marker_color='lightcoral'
        ),
        row=2, col=1
    )
    
    # 4. Statistics table
    stats_data = []
    for col in bedroom_cols:
        stats_data.append([
            col.replace('fmr_', '').upper(),
            f"${ri_fmrs[col].min():.0f}",
            f"${ri_fmrs[col].mean():.0f}",
            f"${ri_fmrs[col].max():.0f}",
            f"${ri_fmrs[col].std():.0f}"
        ])
    
    fig.add_trace(
        go.Table(
            header=dict(
                values=['Type', 'Min', 'Mean', 'Max', 'Std Dev'],
                fill_color='paleturquoise',
                align='left'
            ),
            cells=dict(
                values=list(zip(*stats_data)),
                fill_color='lavender',
                align='left'
            )
        ),
        row=2, col=2
    )
    
    # Update layout
    fig.update_xaxes(title_text="FMR ($)", row=1, col=2)
    fig.update_yaxes(title_text="Area", row=1, col=2)
    fig.update_xaxes(title_text="Bedroom Type", row=2, col=1)
    fig.update_yaxes(title_text="Average FMR ($)", row=2, col=1)
    
    fig.update_layout(
        title_text="Rhode Island Fair Market Rents Dashboard - 2023",
        showlegend=True,
        height=800,
        template='plotly_white'
    )
    
    fig.show()
    
    print(f"✓ Dashboard created with {len(ri_fmrs)} areas")
else:
    print("⚠️  No data available. Please run the previous cell to fetch FMR data.")

## 3. Visualize FMR Distribution

Comprehensive dashboard of Fair Market Rents across Rhode Island.

In [ ]:
# Get Rhode Island FMRs for 2023
try:
    ri_fmrs = hud.get_state_fmrs('RI', year=2023)
    print(f"✓ Retrieved FMR data for {len(ri_fmrs)} areas")
    print(f"\nColumns: {ri_fmrs.columns.tolist()}")
    print(f"\nData Types:")
    print(f"  Metro areas: {len(ri_fmrs[ri_fmrs['county_name'].isna()])}")
    print(f"  Counties/Towns: {len(ri_fmrs[ri_fmrs['county_name'].notna()])}")
    ri_fmrs.head()
except Exception as e:
    print(f"✗ Error fetching data: {e}")
    print("\nMake sure you have set your HUD_API_KEY environment variable")
    print("Get your free key: https://www.huduser.gov/hudapi/public/register?comingfrom=2,3")

## 5. Load FMR Data from File

Alternative: Load downloaded FMR data from CSV/Excel.

In [ ]:
# If you prefer to use downloaded files
# Download from: https://www.huduser.gov/portal/datasets/fmr.html

# fmr_file = "path/to/FY2023_FMRs.xlsx"
# fmr_data = hud.load_fmr_data(fmr_file)
# print(f"Loaded {len(fmr_data)} FMR records")

## 6. Filter by Bedroom Count

Extract FMRs for specific bedroom counts.

In [ ]:
if 'ri_fmrs' in locals() and not ri_fmrs.empty:
    # Get 2-bedroom FMRs
    fmr_2br_data = ri_fmrs[['county_name', 'metro_name', 'fmr_2br']].copy()
    fmr_2br_data['area'] = fmr_2br_data.apply(
        lambda x: x['county_name'] if pd.notna(x['county_name']) else x['metro_name'],
        axis=1
    )
    
    print("2-Bedroom Fair Market Rents:")
    print(fmr_2br_data[['area', 'fmr_2br']].head(10))
    
    # Summary statistics
    print(f"\n📊 2BR FMR Statistics:")
    print(f"  Average: ${ri_fmrs['fmr_2br'].mean():.0f}")
    print(f"  Median: ${ri_fmrs['fmr_2br'].median():.0f}")
    print(f"  Min: ${ri_fmrs['fmr_2br'].min():.0f}")
    print(f"  Max: ${ri_fmrs['fmr_2br'].max():.0f}")
    print(f"  Std Dev: ${ri_fmrs['fmr_2br'].std():.0f}")
else:
    print("⚠️  No data available. Run cell 2 first to fetch FMR data.")

## 7. Calculate Affordability

Determine if housing is affordable based on income (30% rule).

In [ ]:
# Example: Household earning $50,000/year
household_income = 50000
typical_fmr = 1200  # 2BR FMR
bedrooms = 2

affordability = hud.calculate_affordability(
    income=household_income,
    bedrooms=bedrooms,
    fmr_value=typical_fmr,
    income_threshold=0.30
)

print(f"Household Income: ${household_income:,}")
print(f"FMR ({bedrooms}BR): ${typical_fmr}")
print(f"\nMax Affordable Rent: ${affordability['max_affordable_rent']:.2f}")
print(f"Rent-to-Income Ratio: {affordability['rent_to_income_ratio']:.1f}%")
print(f"Affordable: {affordability['is_affordable']}")
print(f"Monthly Surplus/Deficit: ${affordability['monthly_surplus_deficit']:.2f}")

## 8. Compare Counties

Analyze FMR differences across counties.

In [ ]:
if 'ri_fmrs' in locals() and not ri_fmrs.empty:
    # Get county data (exclude metros)
    county_data = ri_fmrs[ri_fmrs['county_name'].notna()].copy()
    
    if not county_data.empty:
        # Group by county and get average FMRs
        county_comparison = county_data.groupby('county_name').agg({
            'fmr_2br': ['mean', 'min', 'max', 'count']
        }).round(0)
        
        county_comparison.columns = ['Avg 2BR', 'Min 2BR', 'Max 2BR', 'Areas']
        county_comparison = county_comparison.sort_values('Avg 2BR', ascending=False)
        
        print("📊 County Comparison (2BR FMRs):")
        print(county_comparison)
        
        # Visualize
        fig = go.Figure()
        
        fig.add_trace(go.Bar(
            x=county_comparison.index,
            y=county_comparison['Avg 2BR'],
            text=county_comparison['Avg 2BR'].astype(int),
            textposition='auto',
            marker_color='steelblue',
            name='Average 2BR FMR'
        ))
        
        fig.update_layout(
            title='Average 2-Bedroom FMR by County',
            xaxis_title='County',
            yaxis_title='FMR ($)',
            height=400,
            template='plotly_white'
        )
        
        fig.show()
    else:
        print("No county data available")
else:
    print("⚠️  No data available. Run cell 2 first to fetch FMR data.")

## 9. Metro Area Deep Dive

Detailed analysis of metro areas with visualizations.

In [ ]:
if 'ri_fmrs' in locals() and not ri_fmrs.empty:
    # Get metro area data
    metros = ri_fmrs[ri_fmrs['county_name'].isna()].copy()
    
    if not metros.empty:
        print(f"Found {len(metros)} metro areas in Rhode Island:\n")
        
        # Create comparison chart
        bedroom_cols = ['fmr_0br', 'fmr_1br', 'fmr_2br', 'fmr_3br', 'fmr_4br']
        
        fig = go.Figure()
        
        for _, metro in metros.iterrows():
            fig.add_trace(go.Bar(
                name=metro['metro_name'],
                x=['0BR', '1BR', '2BR', '3BR', '4BR'],
                y=[metro[col] for col in bedroom_cols],
                text=[f"${metro[col]:.0f}" for col in bedroom_cols],
                textposition='auto'
            ))
        
        fig.update_layout(
            title='Metro Area FMR Comparison - All Bedroom Types',
            xaxis_title='Bedroom Count',
            yaxis_title='FMR ($)',
            barmode='group',
            height=500,
            template='plotly_white',
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1
            )
        )
        
        fig.show()
        
        # Print detailed stats
        print("\n📊 Metro Area Details:")
        for _, metro in metros.iterrows():
            print(f"\n{metro['metro_name']}:")
            for col in bedroom_cols:
                br_type = col.replace('fmr_', '').upper()
                print(f"  {br_type}: ${metro[col]:.0f}")
    else:
        print("No metro area data available")
else:
    print("⚠️  No data available. Run cell 2 first to fetch FMR data.")

## 10. Year-over-Year Analysis

Calculate FMR changes between years (requires fetching multiple years).

In [ ]:
# Get FMRs for two consecutive years
fmr_2023 = hud.get_state_fmrs('RI', year=2023)
fmr_2022 = hud.get_state_fmrs('RI', year=2022)

# Calculate year-over-year change for 2BR
yoy_change = hud.calculate_yoy_change(fmr_2023, fmr_2022, 'fmr_2br')

print("Year-over-Year FMR Changes (2BR):")
print(yoy_change[['countyname', 'fmr_2br_current', 'fmr_2br_previous', 
                   'yoy_change', 'yoy_change_pct']].round(2))

## 11. Affordability Across Income Levels

Test multiple income scenarios.

In [ ]:
# Test multiple income levels
income_levels = [30000, 40000, 50000, 60000, 75000]
fmr_value = 1200  # 2BR FMR
bedrooms = 2

print(f"Affordability Analysis ({bedrooms}BR FMR: ${fmr_value}):\n")
for income in income_levels:
    result = hud.calculate_affordability(
        income=income,
        bedrooms=bedrooms,
        fmr_value=fmr_value
    )
    status = "✓ Affordable" if result['is_affordable'] else "✗ Not Affordable"
    print(f"${income:,}: {status} (Max: ${result['max_affordable_rent']:.0f})")

## 12. Export Results

Save FMR data for further analysis.

In [ ]:
# Export state FMRs
hud.export_to_csv(ri_fmrs, 'ri_fmrs_2023.csv')

# Export comparison
if not comparison.empty:
    hud.export_to_csv(comparison, 'ri_county_comparison.csv')

print("✓ Data exported successfully")

## Next Steps

**Explore More:**
- Filter by metro area: `get_metro_fmrs()`
- Get income limits: `get_income_limits()`
- Compare across states
- Analyze Small Area FMRs (ZIP level)

**Resources:**
- [HUD USER API Documentation](https://www.huduser.gov/portal/dataset/fmr-api.html)
- [FMR Documentation](https://www.huduser.gov/portal/datasets/fmr.html)
- [Income Limits](https://www.huduser.gov/portal/datasets/il.html)

**Use Cases:**
- Housing voucher program analysis
- Affordable housing site selection
- Regional cost-of-living comparisons
- Policy impact assessment